In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("hive-queries") \
    .master("spark://spark-master:7077")\
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .enableHiveSupport() \
    .getOrCreate()

# spark.stop()

In [21]:
#Use WES Database
spark.sql("USE wes")
current_db = spark.sql("SELECT current_database()").collect()[0][0]
print(current_db)

wes


In [8]:
# from pyspark.sql import SparkSession, Window
# from pyspark.sql.functions import last, first
# import pyspark.sql.functions as F
# # Spark session & context
# spark = (
#     SparkSession.builder.appName("world-energy-stats")
#     .master("spark://spark-master:7077")
#     .config("hive.metastore.uris", "thrift://hive-metastore:9083")
#     .enableHiveSupport()
#     .getOrCreate()
# )

# # Read the dataset from HDFS
# df = (
#     spark.read.format("csv")
#     .option("header", "true")
#     .option("inferSchema", "true")
#     .load("hdfs://namenode:9000/energy-data/owid-energy-data.csv")
# )


# df = df.filter(df["iso_code"].isNotNull())
# df = df[df["year"] >= 1990]

# # Dropping irrelevant columns
# cols_to_drop = [
#     col
#     for col in df.columns
#     if "_per_gdp"
#     if "_per_capita" in col or "_change_pct" in col or "_change_twh" in col
# ]
# df = df.drop(*cols_to_drop)

# # Forward fill missing data
# temp_column = [column for column in df.columns if "year" not in column]
# temp_column = [column for column in temp_column if "country" not in column]
# temp_column

# # Define the windows for forward fill and backward fill
# ffill_window = "(partition by country order by year rows between unbounded preceding and current row)"
# # bfill_window = "(partition by country order by year rows between current row and unbounded following)"

# for col in temp_column:
#     df = df.withColumn(
#         col, F.expr(f"case when isnan({col}) then null else {col} end")
#     ).withColumn(col, F.expr(f"coalesce({col}, last({col}, true) over {ffill_window})"))
#     # .withColumn(col, F.expr(f"coalesce({col}, first({col}, true) over {bfill_window})")))

    
df.toPandas().describe()

,year,population,gdp,biofuel_consumption,biofuel_electricity,biofuel_share_elec,biofuel_share_energy,carbon_intensity_elec,coal_consumption,coal_electricity,...,renewables_share_elec,renewables_share_energy,solar_consumption,solar_electricity,solar_share_elec,solar_share_energy,wind_consumption,wind_electricity,wind_share_elec,wind_share_energy
count,7043.000000,7.011000e+03,5.351000e+03,1567.000000,5057.000000,5034.000000,1530.000000,4707.000000,2540.000000,5300.000000,...,5527.000000,2366.000000,2607.000000,5616.000000,5527.000000,2366.000000,2607.000000,5616.000000,5527.000000,2366.000000
mean,2005.850206,3.114626e+07,4.843148e+11,10.335646,1.708074,2.026834,0.467628,442.166952,455.610511,45.172138,...,30.312188,11.778030,6.152919,1.076653,0.659706,0.267962,15.351554,2.691295,1.411243,0.893753
std,9.361943,1.248185e+08,1.607484e+12,41.977236,7.712350,5.575945,0.878323,221.604080,2019.973032,279.866109,...,32.292499,13.983704,42.185779,10.929859,2.238201,0.753777,87.692953,23.000870,4.852819,2.350120
min,1990.000000,1.833000e+03,2.571720e+08,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1998.000000,6.612955e+05,1.928354e+10,0.000000,0.000000,0.000000,0.000000,270.267000,7.659750,0.000000,...,1.422000,2.034750,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2006.000000,5.458688e+06,6.401920e+10,0.487000,0.000000,0.000000,0.072000,489.510000,40.031000,0.000000,...,17.029000,6.440000,0.004000,0.000000,0.000000,0.001000,0.032000,0.000000,0.000000,0.012000
75%,2014.000000,1.972479e+07,3.009124e+11,3.931000,0.340000,1.402000,0.697750,634.597000,182.572500,6.029500,...,54.375000,17.304000,0.282000,0.010000,0.073000,0.056000,2.693000,0.050000,0.156500,0.514250
max,2022.000000,1.425894e+09,1.815162e+13,433.866000,172.130000,71.429000,7.486000,1000.000000,24559.486000,5421.190000,...,100.000000,74.302000,1115.113000,420.350000,40.000000,7.586000,1988.450000,800.520000,56.840000,26.157000


In [3]:
from utils import run_spark_sql

In [11]:
run_spark_sql("1_energy_overview.sql","1_energy_overview.csv")

+----+----------------+
|year|PRIM_ENERGY_CONS|
+----+----------------+
|1990|        94762.63|
|1991|        95531.11|
|1992|        96604.67|
|1993|        97426.49|
|1994|        98706.89|
|1995|       101034.98|
|1996|       103924.71|
|1997|       104994.93|
|1998|       105593.14|
|1999|       107513.06|
|2000|       110397.99|
|2001|       111476.55|
|2002|       113880.28|
|2003|        117901.2|
|2004|       123710.62|
|2005|       127836.38|
|2006|       131628.22|
|2007|       135648.38|
|2008|       137263.93|
|2009|       134974.75|
+----+----------------+
only showing top 20 rows



In [20]:
run_spark_sql("2_energy_consumption_top15.sql","2_energy_consumption_top15.csv")

+--------------+--------+--------------------------+----+
|       country|iso_code|primary_energy_consumption|RANK|
+--------------+--------+--------------------------+----+
|         China|     CHN|                  43873.07|   1|
| United States|     USA|                 25945.025|   2|
|         India|     IND|                  9584.976|   3|
|        Russia|     RUS|                   8745.49|   4|
|         Japan|     JPN|                  4983.889|   5|
|        Canada|     CAN|                  3843.694|   6|
|        Brazil|     BRA|                   3570.67|   7|
|       Germany|     DEU|                  3549.456|   8|
|   South Korea|     KOR|                  3489.739|   9|
|          Iran|     IRN|                  3360.935|  10|
|  Saudi Arabia|     SAU|                  2987.546|  11|
|        France|     FRA|                   2611.23|  12|
|        Mexico|     MEX|                  2219.266|  13|
|     Indonesia|     IDN|                  2154.942|  14|
|United Kingdo

In [18]:
run_spark_sql("2_energy_consumption_pct_top15.sql", "2_energy_consumption_pct_top15.csv") 

+------------------------------------------------------------------+
|round(((sum(primary_energy_consumption) / total_energy) * 100), 2)|
+------------------------------------------------------------------+
|                                                             74.17|
+------------------------------------------------------------------+



In [17]:
run_spark_sql("2_energy_consumption_pct_rem.sql", "2_energy_consumption_pct_rem.csv") 

+--------+
|count(1)|
+--------+
|     129|
+--------+



In [31]:
run_spark_sql("3_energy_breakdown_top15.sql", "3_energy_breakdown_top15.csv") 

+----+--------------------+---------------------+--------------------+------------------------+----------------------+------------------------+--------------------+----------------------+---------------------+
|year|perc_ren_consumption|perc_coal_consumption|perc_gas_consumption|perc_biofuel_consumption|perc_hydro_consumption|perc_nuclear_consumption|perc_oil_consumption|perc_solar_consumption|perc_wind_consumption|
+----+--------------------+---------------------+--------------------+------------------------+----------------------+------------------------+--------------------+----------------------+---------------------+
|1990|   6.603875227748031|   28.674992256085947|   19.93281372952673|     0.13608215427031167|     6.048725873619321|       6.778699498617638|   38.41397725982804|  0.001674441043321...| 0.013034787008694346|
|1991|   6.666393152711433|   28.233939151932958|  20.343556732027615|     0.13990238810267616|     6.090433489009183|      7.0374913576502065|   38.13846895774

In [7]:
run_spark_sql("4_electricity_gen_top15.sql", "4_electricity_gen_top15.csv") 


+--------------+----+------------------------+------------------+------------------------+--------------------+
|       country|year|total_fossil_electricity| total_consumption|non_renewable_percentage|renewable_percentage|
+--------------+----+------------------------+------------------+------------------------+--------------------+
|         China|2021|                 5623.99|          8242.446|                   68.23|               31.77|
| United States|2021|                 2512.39|           3446.46|                    72.9|                27.1|
|         India|2021|                 1337.63|1706.5900000000001|                   78.38|               21.62|
|         Japan|2021|                  680.58|            937.24|                   72.62|               27.38|
|        Russia|2021|                   666.3| 888.8499999999999|                   74.96|               25.04|
|        Brazil|2021|                  139.21|            705.48|                   19.73|              

In [18]:
run_spark_sql("4_electricity_share_top15.sql", "4_electricity_share_top15.csv") 

+--------------+----+-----------------+---------------------+------------------+----------------------+---------+--------------------+
|       country|year|fossil_share_elec|renewables_share_elec|electricity_demand|electricity_generation|max_share|      max_share_name|
+--------------+----+-----------------+---------------------+------------------+----------------------+---------+--------------------+
|         China|2021|           66.289|               28.908|           8466.32|               8484.02|   62.932|     coal_share_elec|
| United States|2021|           60.487|               20.743|           4192.93|               4153.62|   39.513|low_carbon_share_...|
|         India|2021|           78.053|               19.384|           1711.87|               1713.75|   74.173|     coal_share_elec|
|        Russia|2021|           60.008|               19.959|           1092.69|               1110.36|    41.99|      gas_share_elec|
|         Japan|2021|           71.002|               2

In [16]:
# Assuming 'input_year' is the variable that stores the user input for the year
input_year = 2021  # You can replace this with the actual input from the user

# Using the provided syntax
columnNames = spark.sql("SELECT * FROM combined_energy_data").columns

# Building the query string
share_elec_columns_query = f"""
    SELECT
      country,
      electricity_generation,
      year,
      fossil_share_elec,
      renewables_share_elec
    FROM
      combined_energy_data
    WHERE
      year = {input_year}
    ORDER BY
      renewables_share_elec DESC,electricity_generation DESC ,renewables_share_elec  DESC
"""

# Running the query and displaying the results
share_elec_columns_data = spark.sql(share_elec_columns_query)
share_elec_columns_data.show()


+--------------------+----------------------+----+-----------------+---------------------+
|             country|electricity_generation|year|fossil_share_elec|renewables_share_elec|
+--------------------+----------------------+----+-----------------+---------------------+
|             Iceland|                 19.26|2021|              0.0|                100.0|
|              Bhutan|                   9.0|2021|              0.0|                100.0|
|             Albania|                  8.96|2021|              0.0|                100.0|
|               Nepal|                  6.12|2021|              0.0|                100.0|
|             Lesotho|                   0.5|2021|              0.0|                100.0|
|Central African R...|                  0.15|2021|              0.0|                100.0|
|            Ethiopia|                 14.69|2021|            0.068|               99.932|
|            Paraguay|                 40.02|2021|            0.175|               99.825|

In [26]:
run_spark_sql("5_population_correlation.sql", "5_population_correlation.csv") 

+--------------+----------+------------------+--------------------+
|       country|population|    gdp_per_capita|   energy_per_capita|
+--------------+----------+------------------+--------------------+
|        Brazil| 214326224| 13835.92472424653|1.665997717572815...|
|        Canada|  38155012|43753.138975713075|1.007389016153369...|
|         China|1425893504|  12729.9971308264|3.076882661778365E-5|
|        France|  64531448|40006.444668280186|4.046445695748218...|
|       Germany|  83408560| 46589.47851760059|4.255505669921648...|
|         India|1407563904| 6277.340219829905|6.809620488818673E-6|
|     Indonesia| 273753184|11234.412186767478| 7.87184268877764E-6|
|          Iran|  87923432|15911.525649089766|3.822570301850819...|
|         Japan| 124612528| 39057.15316816299|3.999508781332163E-5|
|        Mexico| 126705136| 15836.76709957519|1.751520159372229...|
|        Russia| 145102752|24902.425418147825|6.027101401908628E-5|
|  Saudi Arabia|  35950396| 46657.02729994963|8.